### Step 0 (A). Run this cell if running locally.

In [20]:
# !rm -rf outputs
# !mkdir outputs
# !mkdir outputs/train_outputs
# !mkdir outputs/traveler_outputs

### Step 0 (B). Run this cell if using Google Colab.

In [21]:
# !rm -rf *
# !mkdir outputs
# !mkdir outputs/train_outputs
# !mkdir outputs/traveler_outputs

# !wget "https://raw.githubusercontent.com/cb-cities/transit_sim/TRB/working_example_beijing_subway/inputs/gtfs_line6_stop_times.csv" -O inputs/gtfs_line6_stop_times.csv
# !wget "https://raw.githubusercontent.com/cb-cities/transit_sim/TRB/working_example_beijing_subway/inputs/gtfs_line6_stops.csv" -O inputs/gtfs_line6_stops.csv
# !wget "https://raw.githubusercontent.com/cb-cities/transit_sim/TRB/working_example_beijing_subway/inputs/gtfs_line6_trips.csv" -O inputs/gtfs_line6_trips.csv
# !wget "https://raw.githubusercontent.com/cb-cities/transit_sim/TRB/working_example_beijing_subway/inputs/beijing_line6_od.csv" -O inputs/beijing_line6_od.csv

# !mkdir model
# !wget "https://raw.githubusercontent.com/cb-cities/transit_sim/TRB/model/transit_sim_model.py" -O model/transit_sim_model.py

# !mkdir sp
# !wget "https://github.com/UCB-CE170a/Fall2021/raw/master/traffic_data/liblsp.so" -O sp/liblsp.so
# !wget "https://raw.githubusercontent.com/UCB-CE170a/Fall2021/master/traffic_data/interface.py" -O sp/interface.py

# !pip install geopandas

### Step 1. Import required Python packages
Required packages (installation instructions provided in README.md):
 * Generic: numpy
 * Geometry manipulation: shapely, geopandas
 * Shortest path: sp (https://github.com/cb-cities/sp)
 * Plotting: matplotlib

In [4]:
### plotting
import pandas as pd 
import matplotlib.pyplot as plt

### user module
import sys
sys.path.append('..')
from model.transit_sim_model import Network, Trains, Travelers

### fix random seed
import numpy as np
np.random.seed(0)

### Step 2. Process GTFS schedules
Need the following tables:
 * stop_times.txt: schedule info
 * trips.txt: map trip_id to route_id
 * stops.txt: get stop coordinates (for visualization)

In [14]:
### input and output path
in_path = 'inputs' ### replace with path to input folder
out_path = 'outputs' ### replace with path to output folder
start_time = 5*3600 ### simulation start time in seconds since 12 AM
end_time = 11*3600 ### simulation end time in seconds  since 12 AM 
time_step = 20 ### simulation time step size in seconds

### read in GTFS files
stop_times_file = '{}/gtfs_line6_stop_times.csv'.format(in_path)
trips_file = '{}/gtfs_line6_trips.csv'.format(in_path)
stops_file = '{}/gtfs_line6_stops.csv'.format(in_path)

### only keep results with this service id
service_id = 'weekday' ### GTFS service_id, can be found in calendar.txt file
scen_nm = 'beijing_line6'

### create all trains from GTFS
all_trains = Trains()
all_nodes, all_links = all_trains.schedule_and_network_from_gtfs(
    stop_times_file, trips_file, stops_file, service_id)

### create network from nodes and links
network = Network(all_nodes, all_links)

In [15]:
### display and export schedule
all_trains.schedule_df.to_csv('{}/{}_schedule.csv'.format(out_path, scen_nm), index=False)
all_trains.schedule_df.head(1)

### display and export network
display(network.all_nodes.head(1))
display(network.all_links.head(1))
network.all_links.to_csv('{}/{}_links.csv'.format(out_path, scen_nm), index=False)
network.all_nodes.to_csv('{}/{}_nodes.csv'.format(out_path, scen_nm), index=False)

,route_stop_id,stop_lon,stop_lat,stop_id,type,node_id,geometry
0,downward-caofang,3289.08143,-1651.531166,caofang,platform,0,POINT (3289.081 -1651.531)


,route_stop_id,next_route_stop_id,start_nid,end_nid,initial_weight,geometry
2941,downward-caofang,downward-changying,0,1,1.0,"LINESTRING (3289.081 -1651.531, 3212.888 -1652..."


### Step 4. Travel demand
 * random demand
 * or, input csv with columns *traveler_id*, *origin_nid*, *destin_nid*, *departure_time*

In [16]:
def set_demand(scen_nm, network):
    travelers_df = pd.read_csv('{}/{}_od.csv'.format(in_path, scen_nm))

    station_to_node_dict = {getattr(row, 'stop_id'): getattr(row, 'node_id') for row in network.all_nodes.itertuples()}
    travelers_df['origin_nid'] = travelers_df['enter_station'].map(station_to_node_dict)#.astype(int)
    travelers_df['destin_nid'] = travelers_df['exit_station'].map(station_to_node_dict)#.astype(int)
    travelers_df['traveler_id'] = np.arange(travelers_df.shape[0])
    travelers_df = travelers_df[travelers_df['origin_nid'] != travelers_df['destin_nid']]

    travelers = Travelers()
    travelers.travelers_df = travelers_df[['traveler_id', 'origin_nid', 'destin_nid', 'departure_time']].copy()#.iloc[1871:1875]
    travelers.find_routes(network.network_g, network.station_id_nm_dict, network.station_id_route_dict)
    travelers.set_initial_status(network.station_id_nm_dict)

    print(travelers.travelers_df.shape)
    display(travelers.travelers_df.tail())
    
    return travelers

### Step 5. Run the simulation

In [17]:
def save_agg_results(network, trains, travelers, t, scen_nm=''):
    ### save train results
    train_positions = trains.get_all_train_positions(network)
    train_positions.to_csv('{}/train_outputs/train_outputs_{}_{}.csv'.format(out_path, scen_nm, t), index=False)
    ### save aggregated traveler results
    traveler_locations = travelers.travelers_df[['traveler_status', 'association']].fillna(np.nan).groupby(
            ['traveler_status', 'association']).size().to_frame(
            name='num_travelers').reset_index(drop=False)
    traveler_locations.to_csv('{}/traveler_outputs/agg_traveler_outputs_{}_{}.csv'.format(out_path, scen_nm, t), 
                              index=False)

def save_indiv_results(travelers, t, scen_nm=''):
    ### save individual traveler results
    travelers.travelers_df.to_csv('{}/traveler_outputs/indiv_traveler_outputs_{}_{}.csv'.format(out_path, scen_nm, t), 
                                  index=False)
    
def save_new_board(new_board, t, scen_nm = ''):
    #print(new_board.shape[0])
    if new_board.shape[0]>0:
        new_board.to_csv('{}/traveler_outputs/boarding_traveler_outputs_{}_{}.csv'.format(out_path, scen_nm, t), 
                                  index=False)
        
def run_simulation(network, all_trains, travelers, 
                   train_capacity_pp=1960, train_capacity_sp=1960,
                   transfer_time_pp=40, transfer_time_sp=20,
                   exit_walking_time_pp=40, exit_walking_time_sp=20,
                  ):
    
    t_init, t_end = 3600*5, 3600*12
    trace_ods = [90192, 90526, 91550, 87102, 73120, 57417]
    trace_list = []
    scen_spec = 'cap{}-{}_tt{}-{}'.format(train_capacity_pp, train_capacity_sp, 
                                          transfer_time_pp, transfer_time_sp)

    agent_status = None
    for t in range(t_init, t_end, 20):

        ### update train location
        all_trains.update_location_occupancy(t)

        ### update traveler status
        # new_board = travelers.traveler_update(network, all_trains, t, train_capacity=1e5, transfer_time=0)
        if (t>=7.5*3600) and (t<=8.75*3600):
            new_board = travelers.traveler_update(network, all_trains, t, 
                                                  train_capacity=train_capacity_pp, 
                                                  transfer_time=transfer_time_pp, 
                                                  exit_walking_time=exit_walking_time_pp)
        else:
            new_board = travelers.traveler_update(network, all_trains, t, 
                                                  train_capacity=train_capacity_sp, 
                                                  transfer_time=transfer_time_sp, 
                                                  exit_walking_time=exit_walking_time_sp)

        ### print and plot results
        save_agg_results(network, all_trains, travelers, t, scen_nm=scen_nm+'_'+scen_spec)
        ### save boarding data for analysis
        # save_new_board(new_board, t, scen_nm=scen_nm)

        trace_list.append(travelers.travelers_df.loc[travelers.travelers_df['traveler_id'].isin(trace_ods), 
                                          ['traveler_id', 'update_time', 'traveler_status', 'association']])

        if (t-t_init)%300==0:
            # save_indiv_results(travelers, t, scen_nm=scen_nm+'_'+scen_spec)
            print('Simulation at {}:{:02}am, {}'.format(t//3600, (t%3600)//60, t))
            # print(travelers.travelers_df.shape[0])
        if (t>=27000) and (t<=27660):
            print(travelers.travelers_df.loc[travelers.travelers_df['association']=='downward_lucheng'].shape)

    ### agent trace
    trace_df = pd.concat(trace_list)
    trace_df = trace_df.groupby(['traveler_id', 'traveler_status', 'association']).first().reset_index()
    trace_df.to_csv('{}/traveler_outputs/traveler_trace_{}.csv'.format(
        out_path, scen_nm+'_'+scen_spec), index=False)
    
    ### final traveler status
    travelers.travelers_df.to_csv('{}/traveler_outputs/final_traveler_status_{}.csv'.format(
        out_path, scen_nm+'_'+scen_spec), index=False)

In [18]:
### reset all traveler status to pre-departure
travelers = set_demand(scen_nm, network)

### run simulation and save outputs
run_simulation(network, all_trains, travelers, 
                   train_capacity_pp=1960, train_capacity_sp=1960,
                   transfer_time_pp=90, transfer_time_sp=90,
                   exit_walking_time_pp=90, exit_walking_time_sp=90)

(150725, 10)


,traveler_id,origin_nid,destin_nid,departure_time,traveler_status,update_time,association,next_station_id,init_boarding_time,final_alighting_time
150720,150720,41,36,26731,0,0,-11,-111,10000000.0,10000000.0
150721,150721,38,36,33945,0,0,-11,-111,10000000.0,10000000.0
150722,150722,30,36,34517,0,0,-11,-111,10000000.0,10000000.0
150723,150723,41,36,29090,0,0,-11,-111,10000000.0,10000000.0
150724,150724,40,36,33440,0,0,-11,-111,10000000.0,10000000.0


Simulation at 5:00am, 18000
Simulation at 5:05am, 18300
Simulation at 5:10am, 18600
Simulation at 5:15am, 18900
Simulation at 5:20am, 19200
Simulation at 5:25am, 19500
Simulation at 5:30am, 19800
Simulation at 5:35am, 20100
Simulation at 5:40am, 20400
Simulation at 5:45am, 20700
Simulation at 5:50am, 21000
Simulation at 5:55am, 21300
Simulation at 6:00am, 21600
Simulation at 6:05am, 21900
Simulation at 6:10am, 22200
Simulation at 6:15am, 22500
Simulation at 6:20am, 22800
Simulation at 6:25am, 23100
Simulation at 6:30am, 23400
Simulation at 6:35am, 23700
Simulation at 6:40am, 24000
Simulation at 6:45am, 24300
Simulation at 6:50am, 24600
Simulation at 6:55am, 24900
Simulation at 7:00am, 25200
Simulation at 7:05am, 25500
Simulation at 7:10am, 25800
Simulation at 7:15am, 26100
Simulation at 7:20am, 26400
Simulation at 7:25am, 26700
Simulation at 7:30am, 27000
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
(0, 15)
Simulation a